# Dev Testing

In [0]:
table_name = "dq_dev.lmg_sandbox.config_driven_table_example"

describe_extended_query = """
DESCRIBE EXTENDED
  {table_name}
"""

spark.sql(describe_extended_query.format(table_name=table_name)).show(truncate=False, n=100)

In [0]:
def describe_table_show(spark, fq_table: str):
    df = spark.sql(f"DESCRIBE TABLE EXTENDED {fq_table}")
    print("=== Raw DataFrame Schema ===")
    df.printSchema()
    print("=== Raw DataFrame ===")
    df.show(truncate=False, n=100)
    return df

def dataframe_to_rowdicts(df):
    rows = [row.asDict() for row in df.collect()]
    print("=== Collected Rows ===")
    for r in rows:
        print(r)
    return rows

# Get DataFrame and show it
df = describe_table_show(spark, "dq_dev.lmg_sandbox.config_driven_table_example")

# Convert to Python list of dicts and show those
rows = dataframe_to_rowdicts(df)

In [0]:
from typing import List, Dict, Any

def parse_describe_table(rows: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Convert the output of DESCRIBE TABLE EXTENDED (rows as dicts)
    into a nested dict matching YAML's structure.
    """
    table_level = {}
    columns = []
    partitioned_by = []
    constraints = []
    table_properties = {}
    owner = None
    comment = None

    # State tracking
    mode = "columns"

    for row in rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        comm = (row.get("comment") or "").strip() if row.get("comment") else None

        # Section transitions
        if col_name == "# Partition Information":
            mode = "partition"
            continue
        elif col_name == "# Detailed Table Information":
            mode = "details"
            continue
        elif col_name == "# Constraints":
            mode = "constraints"
            continue
        elif col_name.startswith("#"):
            mode = "skip"
            continue

        if mode == "columns" and col_name and not col_name.startswith("#"):
            columns.append({
                "name": col_name,
                "datatype": data_type,
                "comment": comm if comm and comm.upper() != "NULL" else "",
                # Placeholders for additional fields
                "nullable": None,
                "tags": {},
                "column_masking_rule": None,
                "default_value": None,
                "variable_value": None,
                "allowed_values": [],
                "column_check_constraints": {},
                "active": True,
            })
        elif mode == "partition" and col_name and col_name != "# col_name":
            partitioned_by.append(col_name)
        elif mode == "details":
            if col_name == "Catalog":
                table_level["catalog"] = data_type
            elif col_name == "Database":
                table_level["schema"] = data_type
            elif col_name == "Table":
                table_level["table"] = data_type
            elif col_name == "Owner":
                owner = data_type
            elif col_name == "Comment":
                comment = data_type
            elif col_name == "Table Properties":
                # Parse table properties string into dict
                for prop in data_type.strip("[]").split(","):
                    if "=" in prop:
                        k, v = prop.split("=", 1)
                        table_properties[k.strip()] = v.strip()
            # Add more detail parsing as needed

        elif mode == "constraints" and col_name and data_type:
            constraints.append((col_name, data_type))

    # Compose snapshot
    table_level["owner"] = owner
    table_level["comment"] = comment
    table_level["partitioned_by"] = partitioned_by
    table_level["table_properties"] = table_properties
    # Parse out PK/unique from constraints
    pk = []
    for cname, dtype in constraints:
        if dtype.startswith("PRIMARY KEY"):
            pk.append(dtype.split("`")[1].replace("`", ""))
    table_level["primary_key"] = pk

    # Final structure
    return {
        "table_level_values": table_level,
        "column_level_values": columns,
    }

# ---- Example usage ----
snapshot = parse_describe_table(rows)
from pprint import pprint
pprint(snapshot)

In [0]:
from typing import List, Dict
from pyspark.sql import SparkSession

def parse_fully_qualified_table(fq_table: str):
    """Split catalog.schema.table into (catalog, schema, table)"""
    parts = fq_table.split(".")
    if len(parts) != 3:
        raise ValueError(f"Expected catalog.schema.table, got: {fq_table}")
    return parts[0], parts[1], parts[2]

def spark_sql_to_rows(spark: SparkSession, sql: str) -> List[dict]:
    """Runs a Spark SQL and returns the result as a list of dicts."""
    df = spark.sql(sql)
    return [row.asDict() for row in df.collect()]

def get_table_tags(spark: SparkSession, fq_table: str) -> Dict[str, str]:
    """
    Return all tags for the given table as a dict: {tag_name: tag_value, ...}
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT tag_name, tag_value
        FROM system.information_schema.table_tags
        WHERE catalog_name = '{catalog}'
          AND schema_name = '{schema}'
          AND table_name = '{table}'
    """
    rows = spark_sql_to_rows(spark, sql)
    return {row['tag_name']: row['tag_value'] for row in rows}

def get_column_tags(spark: SparkSession, fq_table: str) -> Dict[str, Dict[str, str]]:
    """
    Return all tags for each column in the table as:
        {column_name: {tag_name: tag_value, ...}, ...}
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT column_name, tag_name, tag_value
        FROM system.information_schema.column_tags
        WHERE catalog_name = '{catalog}'
          AND schema_name = '{schema}'
          AND table_name = '{table}'
    """
    rows = spark_sql_to_rows(spark, sql)
    col_tags = {}
    for row in rows:
        col = row['column_name']
        tag = row['tag_name']
        val = row['tag_value']
        if col not in col_tags:
            col_tags[col] = {}
        col_tags[col][tag] = val
    return col_tags

def get_row_filters(spark: SparkSession, fq_table: str) -> List[dict]:
    """
    Return all row filters for a table as a list of dicts.
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT filter_name, target_columns
        FROM system.information_schema.row_filters
        WHERE table_catalog = '{catalog}'
          AND table_schema = '{schema}'
          AND table_name = '{table}'
    """
    return spark_sql_to_rows(spark, sql)

def get_constraint_table_usage(spark: SparkSession, fq_table: str) -> List[dict]:
    """
    Return all constraints defined on the table (e.g. PK, Unique, FK).
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT constraint_name
        FROM system.information_schema.constraint_table_usage
        WHERE table_catalog = '{catalog}'
          AND table_schema = '{schema}'
          AND table_name = '{table}'
    """
    return spark_sql_to_rows(spark, sql)

def get_constraint_column_usage(spark: SparkSession, fq_table: str) -> List[dict]:
    """
    Return all constraints for all columns on the table.
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT column_name, constraint_name
        FROM system.information_schema.constraint_column_usage
        WHERE table_catalog = '{catalog}'
          AND table_schema = '{schema}'
          AND table_name = '{table}'
    """
    return spark_sql_to_rows(spark, sql)
  
fq = "dq_dev.lmg_sandbox.config_driven_table_example"

  # 1. Table tags
print("=== Table Tags ===")
print(get_table_tags(spark, fq))

# 2. Column tags
print("=== Column Tags ===")
print(get_column_tags(spark, fq))

# 3. Row filters
print("=== Row Filters ===")
print(get_row_filters(spark, fq))

# 4. Table constraints
print("=== Table Constraints ===")
print(get_constraint_table_usage(spark, fq))

# 5. Column constraints
print("=== Column Constraints ===")
print(get_constraint_column_usage(spark, fq))

In [0]:
from typing import List, Dict, Any, Tuple
import re
from pyspark.sql import SparkSession

# --- Spark SQL helpers ---
def spark_sql_to_df(spark: SparkSession, sql: str):
    """
    Run Spark SQL and return the DataFrame. Logs error, re-raises on failure.
    """
    try:
        return spark.sql(sql)
    except Exception as e:
        print(f"[ERROR] spark_sql_to_df failed: {e}\nSQL: {sql}")
        raise

def spark_sql_to_rows(spark: SparkSession, sql: str):
    """
    Run Spark SQL and return results as list of dicts. Logs error, re-raises on failure.
    """
    try:
        df = spark.sql(sql)
        return [row.asDict() for row in df.collect()]
    except Exception as e:
        print(f"[ERROR] spark_sql_to_rows failed: {e}\nSQL: {sql}")
        raise

# --- Table introspection ---
def describe_table_to_rows(spark: SparkSession, fq_table: str) -> List[Dict[str, Any]]:
    sql = f"DESCRIBE TABLE EXTENDED {fq_table}"
    return spark_sql_to_rows(spark, sql)

def extract_columns(spark: SparkSession, fq_table: str) -> List[Dict[str, Any]]:
    rows = describe_table_to_rows(spark, fq_table)
    cols = []
    for row in rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        comm = (row.get("comment") or "").strip() if row.get("comment") else None
        if col_name and not col_name.startswith("#"):
            cols.append({
                "name": col_name,
                "datatype": data_type,
                "comment": comm if comm and comm.upper() != "NULL" else "",
                "nullable": None,
                "tags": {},
                "column_masking_rule": None,
                "default_value": None,
                "variable_value": None,
                "allowed_values": [],
                "column_check_constraints": {},
                "active": True,
            })
    return cols

def extract_partitioned_by(spark: SparkSession, fq_table: str) -> List[str]:
    rows = describe_table_to_rows(spark, fq_table)
    collecting = False
    partition_cols = []
    for row in rows:
        col_name = (row.get("col_name") or "").strip()
        if col_name == "# Partition Information":
            collecting = True
            continue
        elif col_name.startswith("#") and collecting:
            break
        elif collecting and col_name and col_name != "# col_name":
            partition_cols.append(col_name)
    return partition_cols

def extract_details(spark: SparkSession, fq_table: str) -> Dict[str, Any]:
    rows = describe_table_to_rows(spark, fq_table)
    details = {}
    props = {}
    owner = None
    comment = None
    in_details = False
    for row in rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        if col_name == "# Detailed Table Information":
            in_details = True
            continue
        if in_details:
            if col_name == "" or col_name.startswith("#"):
                break
            if col_name == "Catalog":
                details["catalog"] = data_type
            elif col_name == "Database":
                details["schema"] = data_type
            elif col_name == "Table":
                details["table"] = data_type
            elif col_name == "Owner":
                owner = data_type
            elif col_name == "Comment":
                comment = data_type
            elif col_name == "Table Properties":
                for prop in data_type.strip("[]").split(","):
                    if "=" in prop:
                        k, v = prop.split("=", 1)
                        props[k.strip()] = v.strip()
    details["owner"] = owner
    details["comment"] = comment
    details["table_properties"] = props
    return details

def extract_constraints(spark: SparkSession, fq_table: str) -> List[Tuple[str, str]]:
    rows = describe_table_to_rows(spark, fq_table)
    constraints = []
    in_constraints = False
    for row in rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        if col_name == "# Constraints":
            in_constraints = True
            continue
        if in_constraints:
            if not col_name or col_name.startswith("#"):
                break
            if col_name and data_type:
                constraints.append((col_name, data_type))
    return constraints

def extract_primary_key(spark: SparkSession, fq_table: str) -> List[str]:
    constraints = extract_constraints(spark, fq_table)
    pk = []
    for cname, dtype in constraints:
        if "PRIMARY KEY" in dtype:
            m = re.search(r"\((.*?)\)", dtype)
            if m:
                cols = [c.strip().replace("`", "") for c in m.group(1).split(",")]
                pk += cols
    return pk

def parse_describe_table(spark: SparkSession, fq_table: str) -> Dict[str, Any]:
    details = extract_details(spark, fq_table)
    columns = extract_columns(spark, fq_table)
    partitioned_by = extract_partitioned_by(spark, fq_table)
    pk = extract_primary_key(spark, fq_table)
    details["partitioned_by"] = partitioned_by
    details["primary_key"] = pk
    return {
        "table_level_values": details,
        "column_level_values": columns,
    }

# ---- TEST ALL FUNCTIONS ----

fq_table = "dq_dev.lmg_sandbox.config_driven_table_example"   # <--- update as needed

print("==== COLUMNS ====")
print(extract_columns(spark, fq_table))

print("==== PARTITIONED BY ====")
print(extract_partitioned_by(spark, fq_table))

print("==== DETAILS ====")
print(extract_details(spark, fq_table))

print("==== CONSTRAINTS ====")
print(extract_constraints(spark, fq_table))

print("==== PRIMARY KEY ====")
print(extract_primary_key(spark, fq_table))

print("==== FULL SNAPSHOT ====")
from pprint import pprint
pprint(parse_describe_table(spark, fq_table))

In [0]:
def describe_table_to_rows(spark, fq_table: str):
    """Run DESCRIBE TABLE EXTENDED and return rows as list of dicts."""
    df = spark.sql(f"DESCRIBE TABLE EXTENDED {fq_table}")
    return [row.asDict() for row in df.collect()]


from typing import List, Dict, Any, Optional
import re

def extract_columns(describe_rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Extract just the column definitions from DESCRIBE output."""
    columns = []
    for row in describe_rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        comment = (row.get("comment") or "").strip() if row.get("comment") else None

        # Columns section (stops at # Partition Info)
        if col_name == "" or col_name.startswith("#"):
            if col_name == "# Partition Information":
                break
            continue
        columns.append({
            "name": col_name,
            "datatype": data_type,
            "comment": comment if comment and comment.upper() != "NULL" else "",
            # only these three fields are real here
        })
    return columns

def extract_partitioned_by(describe_rows: List[Dict[str, Any]]) -> List[str]:
    """Extract partition columns (if any)."""
    collecting = False
    partition_cols = []
    for row in describe_rows:
        col_name = (row.get("col_name") or "").strip()
        if col_name == "# Partition Information":
            collecting = True
            continue
        if collecting:
            if not col_name or col_name.startswith("#"):
                break
            if col_name != "# col_name":
                partition_cols.append(col_name)
    return partition_cols

def extract_table_details(describe_rows: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Extract catalog, schema, table, owner, comment, table_properties only."""
    details = {}
    table_properties = {}
    in_details = False
    for row in describe_rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        if col_name == "# Detailed Table Information":
            in_details = True
            continue
        if in_details:
            if not col_name or col_name.startswith("#"):
                break
            if col_name == "Catalog":
                details["catalog"] = data_type
            elif col_name == "Database":
                details["schema"] = data_type
            elif col_name == "Table":
                details["table"] = data_type
            elif col_name == "Owner":
                details["owner"] = data_type
            elif col_name == "Comment":
                details["comment"] = data_type
            elif col_name == "Table Properties":
                for prop in data_type.strip("[]").split(","):
                    if "=" in prop:
                        k, v = prop.split("=", 1)
                        table_properties[k.strip()] = v.strip()
    details["table_properties"] = table_properties
    return details

def extract_constraints(describe_rows: List[Dict[str, Any]]) -> List[Dict[str, str]]:
    """Extract table constraints as dicts: name/type"""
    constraints = []
    in_constraints = False
    for row in describe_rows:
        col_name = (row.get("col_name") or "").strip()
        data_type = (row.get("data_type") or "").strip()
        if col_name == "# Constraints":
            in_constraints = True
            continue
        if in_constraints:
            if not col_name or col_name.startswith("#"):
                break
            if col_name and data_type:
                constraints.append({"name": col_name, "type": data_type})
    return constraints

def extract_primary_key(describe_rows: List[Dict[str, Any]]) -> Optional[List[str]]:
    """Find and parse PRIMARY KEY constraint, if present."""
    cons = extract_constraints(describe_rows)
    for c in cons:
        if "PRIMARY KEY" in c["type"]:
            m = re.search(r"\((.*?)\)", c["type"])
            if m:
                return [col.strip().replace("`", "") for col in m.group(1).split(",")]
    return None


###########################################################################


from typing import List, Dict
from pyspark.sql import SparkSession

def parse_fully_qualified_table(fq_table: str):
    """Split catalog.schema.table into (catalog, schema, table)"""
    parts = fq_table.split(".")
    if len(parts) != 3:
        raise ValueError(f"Expected catalog.schema.table, got: {fq_table}")
    return parts[0], parts[1], parts[2]

def spark_sql_to_rows(spark: SparkSession, sql: str) -> List[dict]:
    """Runs a Spark SQL and returns the result as a list of dicts."""
    df = spark.sql(sql)
    return [row.asDict() for row in df.collect()]

def get_table_tags(spark: SparkSession, fq_table: str) -> Dict[str, str]:
    """
    Return all tags for the given table as a dict: {tag_name: tag_value, ...}
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT tag_name, tag_value
        FROM system.information_schema.table_tags
        WHERE catalog_name = '{catalog}'
          AND schema_name = '{schema}'
          AND table_name = '{table}'
    """
    rows = spark_sql_to_rows(spark, sql)
    return {row['tag_name']: row['tag_value'] for row in rows}

def get_column_tags(spark: SparkSession, fq_table: str) -> Dict[str, Dict[str, str]]:
    """
    Return all tags for each column in the table as:
        {column_name: {tag_name: tag_value, ...}, ...}
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT column_name, tag_name, tag_value
        FROM system.information_schema.column_tags
        WHERE catalog_name = '{catalog}'
          AND schema_name = '{schema}'
          AND table_name = '{table}'
    """
    rows = spark_sql_to_rows(spark, sql)
    col_tags = {}
    for row in rows:
        col = row['column_name']
        tag = row['tag_name']
        val = row['tag_value']
        if col not in col_tags:
            col_tags[col] = {}
        col_tags[col][tag] = val
    return col_tags

def get_row_filters(spark: SparkSession, fq_table: str) -> List[dict]:
    """
    Return all row filters for a table as a list of dicts.
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT filter_name, target_columns
        FROM system.information_schema.row_filters
        WHERE table_catalog = '{catalog}'
          AND table_schema = '{schema}'
          AND table_name = '{table}'
    """
    return spark_sql_to_rows(spark, sql)

def get_constraint_table_usage(spark: SparkSession, fq_table: str) -> List[dict]:
    """
    Return all constraints defined on the table (e.g. PK, Unique, FK).
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT constraint_name
        FROM system.information_schema.constraint_table_usage
        WHERE table_catalog = '{catalog}'
          AND table_schema = '{schema}'
          AND table_name = '{table}'
    """
    return spark_sql_to_rows(spark, sql)

def get_constraint_column_usage(spark: SparkSession, fq_table: str) -> List[dict]:
    """
    Return all constraints for all columns on the table.
    """
    catalog, schema, table = parse_fully_qualified_table(fq_table)
    sql = f"""
        SELECT column_name, constraint_name
        FROM system.information_schema.constraint_column_usage
        WHERE table_catalog = '{catalog}'
          AND table_schema = '{schema}'
          AND table_name = '{table}'
    """
    return spark_sql_to_rows(spark, sql)

In [0]:
# --- Run these in order to see exactly what you get ---
fq = "dq_dev.lmg_sandbox.config_driven_table_example"
rows = describe_table_to_rows(spark, fq)

print("\n--- Columns ---")
print(extract_columns(rows))

print("\n--- Partitioned By ---")
print(extract_partitioned_by(rows))

print("\n--- Table Details ---")
print(extract_table_details(rows))

print("\n--- Constraints ---")
print(extract_constraints(rows))

print("\n--- Primary Key ---")
print(extract_primary_key(rows))


###################################################################


# 1. Table tags
print("=== Table Tags ===")
print(get_table_tags(spark, fq))

# 2. Column tags
print("=== Column Tags ===")
print(get_column_tags(spark, fq))

# 3. Row filters
print("=== Row Filters ===")
print(get_row_filters(spark, fq))

# 4. Table constraints
print("=== Table Constraints ===")
print(get_constraint_table_usage(spark, fq))

# 5. Column constraints
print("=== Column Constraints ===")
print(get_constraint_column_usage(spark, fq))